In [20]:
import pandas as pd
from groq import Groq
import json
import os
import re
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
import logging

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

## load evv variables
from dotenv import load_dotenv

load_dotenv()
GROQ_API_KEY = os.environ["GROQ_API_KEY"]
CHAT_MODEL = os.environ["CHAT_MODEL"]
client = Groq()

c:\Users\SP14267\Desktop\seh\venv\Lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [16]:
df = pd.read_csv("../data/AI_feedback.csv", encoding='utf-8')
df.head()

,Comment,Label
0,The article is poorly researched and full of i...,Negative_Complaint
1,You didn’t cover the key aspects of the topic.,Negative_Complaint
2,The writing is too convoluted and hard to follow.,Negative_Complaint
3,There are too many grammatical errors in the c...,Negative_Complaint
4,Your articles are always biased.,Negative_Complaint


In [19]:
list(df['Label'].unique())

['Negative_Complaint',
 'Unhelpful_Negative',
 'Moderate_Constructive',
 'Detailed_Constructive',
 'Helpful_Positive',
 'Unhelpful_Positive',
 'Neutral_Descriptive',
 'Neutral_Generic',
 'Spam',
 'Emotion_Angry',
 'Emotion_Happy',
 'Emotion_Sad',
 'Emotion_Excited',
 'Suggestion',
 'Content_Specific',
 'Design_Feedback',
 'Constructive_Criticism',
 'Neutral']

In [34]:
df_test = df.sample(20)

In [36]:
comments = list(df_test['Comment'])
labels = list(df_test['Label'])


In [40]:
GENERATE_LABELS_PROMPT = '''
You are an intelligent system tasked with classifying seller feedback for an e-commerce platform. Each feedback item should be categorised into one or more appropriate labels from the following list:
['Negative_Complaint', 'Unhelpful_Negative', 'Moderate_Constructive', 'Detailed_Constructive', 'Helpful_Positive', 'Unhelpful_Positive', 'Neutral_Descriptive', 'Neutral_Generic', 'Spam', 'Emotion_Angry', 'Emotion_Happy', 'Emotion_Sad', 'Emotion_Excited', 'Suggestion', 'Content_Specific', 'Design_Feedback', 'Constructive_Criticism', 'Neutral']

Now classify the following feedback:
Feedback: {text}

Example Output format:
{{
    "feedback": "This platform is very confusing to use. The design needs improvement.",
    "label": "Negative_Complaint"
}}
''' 

In [43]:
def extract_answer(input_string):
    # Find the start and end indices of the JSON data within the input string
    # Assuming the JSON data starts with '{' and ends with '}'
    json_start = input_string.find("{")
    json_end = input_string.rfind("}") + 1

    # If either the start or end index is not found, raise an error
    if json_start == -1 or json_end == -1:
        raise ValueError("Invalid input: No JSON data found.")

    # Extract the substring that potentially contains the JSON data
    json_data = input_string[json_start:json_end]

    try:
        # Attempt to convert the JSON string to a Python dictionary
        data_dict = json.loads(json_data)
        return data_dict

    except json.JSONDecodeError:
        # If JSON decoding fails, search for a JSON object containing the 'questions' key
        # Using regex to match a pattern that includes the 'questions' key
        pattern = r'\{\s*"feedback":\s*".*?",\s*"label":\s*".*?"\s*\}'
        match = re.search(pattern, input_string, re.DOTALL)

        if match:
            # If a match is found, extract the matched JSON string and convert it to a dictionary
            data_json_str = match.group(0)
            data_dict = json.loads(data_json_str)
            return data_dict

        # If no valid JSON is found, the function will Log an error
        else:
            logging.error(
                "No dictionary with 'questions' as a key found in this input string. Error by LLM"
            )
            return {"error": "No dictionary with questions found"}


def generate_labels(feedback, label_prompt, client):
    prompt = PromptTemplate(
        template=label_prompt,
        input_variables=["text"],
    )

    final_prompt = prompt.format(text=feedback)

    # Generate the completion by interacting with the language model API
    completion = client.chat.completions.create(
        model=CHAT_MODEL,
        messages=[
                    {
                        "role": "user", 
                        "content": final_prompt
                    }
                    ],
        temperature=0,  # Control the randomness of the output (lower means less random)
        max_tokens=1024,  # Limit the response length
        top_p=1,  # Nucleus sampling parameter (1 means only the most likely tokens are considered)
        stream=True,  # Enable streaming of the response chunks
        stop=None,  # Define stopping conditions (None means no stopping condition)
    )

    # Initialize an empty string to accumulate the response content
    answer = """"""
    for chunk in completion:
        # Append each chunk of content to the answer string
        answer += chunk.choices[0].delta.content or ""
        
    label_dict = extract_answer(answer)

    # Return the dictionary containing the generated questions
    return label_dict

In [ ]:
gen_label_dict = generate_labels(comments[0], GENERATE_LABELS_PROMPT, client)
print("-"* 80)
print(f"FEEDBACK: {gen_label_dict['feedback']}")
print("-"* 80)
print(f"AI Generated label: {gen_label_dict['label']}")
print(f"Correct label: {labels[0]}")


2025-01-20 10:56:07,754 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


--------------------------------------------------------------------------------
Feedback: This topic made me realise how much I’ve been struggling with [issue].
--------------------------------------------------------------------------------
AI Generated label: Emotion_Sad
Correct label: Emotion_Sad
